In [63]:
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import os
import re
import requests
import json

load_dotenv()

True

In [69]:
global100_filepath = Path("data/global100_2021.csv")
global100_df = pd.read_csv(global100_filepath)

sri_filepath = Path("data/SRI Funds - Sheet1.csv")
sri_df = pd.read_csv(sri_filepath)

ctw_filepath = Path("data/change_the_world_2020.csv")
ctw_df = pd.read_csv(ctw_filepath)



In [65]:
global100_df.head()

,Rank 2021,Rank 2020,Name,Peer Group (CKIG),Peer Group Rank,Country,Energy Productivity,Energy Productivity Score,Carbon Productivity,Carbon Productivity Score,...,Clean Investment - Weight,Non-male on Board - Weight,Non-male in Executive Management - Weight,Racial Diversity Among Executives - Weight,Racial Diversity on Board of Directors - Weight,Paid Sick Leave,Sustainability Paylink - Weight,Supplier Score - Weight,Peer Group Rank Total,Ticker
0,1,29.0,Schneider Electric SE,Industrial Conglomerates,1,France,13989.0,96.0,90981.0,66.9,...,8,3,3,3,3,3,5,3,53,SBGSF
1,2,1.0,Orsted A/S,Power Generation,1,Denmark,467.0,39.7,5361.0,63.7,...,8,3,3,3,3,3,5,3,244,DOGEF
2,3,9.0,Banco do Brasil SA,Banks and Investment Services,1,Brazil,232205.0,72.0,851966.0,32.5,...,8,3,3,3,3,3,5,3,935,BDORY
3,4,3.0,Neste Oyj,Oil & Gas,1,Finland,553.0,76.6,7765.0,70.1,...,8,3,3,3,3,3,5,3,347,NTOIY
4,5,57.0,Stantec Inc,Consulting and Professional Services,1,Canada,5418.0,20.8,64411.0,18.2,...,8,3,3,3,3,3,5,3,23,STN


In [66]:
tradier_lookup_url = "https://sandbox.tradier.com/v1/markets/search"
tradier_access_token = os.getenv("TRADIER_ACCESS_TOKEN")
headers = {"Accept": "application/json", "Authorization" : f"Bearer {tradier_access_token}"}

In [67]:
def get_matching_tickers(q):
    """
    Return the list of matching tickers for query term.
    """

    query_terms = q.split(" ")
    num_terms = len(query_terms)
    params = {"q": q, "lang": "en"}

    response = requests.get(tradier_lookup_url, headers=headers, params=params)
    response.raise_for_status()
    quotes = response.json()["securities"]

    # No matching results
    if quotes is None:
        if num_terms > 1:
            return get_matching_tickers(" ".join(query_terms[:num_terms-1]))
        else:
            return None

    quotes = quotes["security"]
    if isinstance(quotes, list):
        if len(quotes) > 0:
            return quotes[0]
        return None
    elif isinstance(quotes, dict):
        return quotes
    
    return None

In [71]:

sri_tickers = []
for name in sri_df["Fund"].values:
    result = get_matching_tickers(name)
    if result is None:
        sri_tickers.append(None)
        print(f"     !!{name}: Ticker not found")
    else:
        ticker = result["symbol"]
        sri_tickers.append(ticker)
        print(f"{name}: {ticker}")

sri_df["Ticker"] = sri_tickers


Access Capital Strategies Community Investment Fund[26]: ACCR
     !!AHA Balanced Fund – Institutional Class: Ticker not found
     !!AHA Diversified Equity Fund – Institutional Class: Ticker not found
     !!AHA Diversified Equity Fund – N Class: Ticker not found
     !!AHA Full Maturity Fixed Income Fund – Institutional Class: Ticker not found
     !!AHA Full Maturity Fixed Income Fund – N Class: Ticker not found
     !!AHA Limited Maturity Fixed Income Fund – Institutional Class: Ticker not found
     !!AHA Limited Maturity Fixed Income Fund – N Class: Ticker not found
     !!AHA Socially Responsible Equity I: Ticker not found
     !!AHA Socially Responsible Equity N: Ticker not found
     !!Appleseed Fund[1]: Ticker not found
Ariel Appreciation Fund [2]: ACEZ
Ariel Focus Fund: ACEZ
Ariel Fund: ACEZ
     !!Azzad Ethical Income Fund [3]: Ticker not found
     !!Azzad Ethical Mid Cap Fund: Ticker not found
     !!Brighter Student Fund [4]: Ticker not found
     !!Fund': Ticker not fou

,Fund,Alcohol,Tobacco,Gambling,Defense/ Weapons,Animal Testing,Products/ Services,Environment,Human Rights,Labor Relations,Employment/ Equality,Community Investment,Proxy Voting,Ticker
0,Access Capital Strategies Community Investment...,NS,NS,NS,NS,NS,NS,NS,NS,NS,NS,P,X,ACCR
1,AHA Balanced Fund – Institutional Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
2,AHA Diversified Equity Fund – Institutional Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
3,AHA Diversified Equity Fund – N Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
4,AHA Full Maturity Fixed Income Fund – Institut...,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Vanguard FTSE Social Index Fund [32],X,X,X,X,X,NS,X,X,NS,NS,NS,NS,VWO
112,WVH Ethical Inc. Social Balanced Fund,X,X,R,X,R,P,P,P,P,P,P,V,None
113,WVH Ethical Inc. Social Equity Fund,X,X,R,X,R,P,P,P,R,P,X,V,None
114,WVH Ethical Inc. Green Growth Fund [33],R,R,R,R,R,P,P,NS,NS,NS,NS,V,None


In [73]:
sri_df.head()

,Fund,Alcohol,Tobacco,Gambling,Defense/ Weapons,Animal Testing,Products/ Services,Environment,Human Rights,Labor Relations,Employment/ Equality,Community Investment,Proxy Voting,Ticker
0,Access Capital Strategies Community Investment...,NS,NS,NS,NS,NS,NS,NS,NS,NS,NS,P,X,ACCR
1,AHA Balanced Fund – Institutional Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
2,AHA Diversified Equity Fund – Institutional Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
3,AHA Diversified Equity Fund – N Class,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
4,AHA Full Maturity Fixed Income Fund – Institut...,NS,X,NS,NS,NS,NS,NS,NS,NS,NS,NS,V,None
